In [23]:
#!/usr/bin/env python3
"""
Intraday backtester with:
 - realistic intraday brokerage & regulatory fees (per provided table)
 - next-bar-open entry
 - strict absolute SL/TP (intrabar)
 - VWAP/EMA exit
 - slippage
 - integer shares sizing, explicit margin/borrow accounting
 - clearer trades CSV with detailed columns
"""

from datetime import datetime, time
import pandas as pd
import numpy as np
import yfinance as yf
import pytz
import os
import math

# =========================
# USER CONFIGURATION
# =========================
TICKERS = ['ICICIBANK.NS']


INTERVAL = "15m"
INITIAL_BALANCE = 200000.0       # ₹ per-ticker
LEVERAGE = 5.0                   # 5x
TRADE_SIZE_FRACTION = 1.0        # fraction of buying_power used per trade (1.0 = full)
VWAP_SESSION_START = time(9, 15)
ENTRY_WINDOW_START = time(9, 45)
ENTRY_WINDOW_END = time(14, 0)
MARKET_CLOSE = time(15, 30)
PROFIT_TARGET_ABS = 2000.0       # ₹ target (absolute)
LOSS_LIMIT_ABS = -500.0          # ₹ stop loss (absolute, negative)
SAVE_FOLDER = "./backtest_results_with_fees"
# Slippage (still applied; brokerage & regulatory fees calculated separately)
SLIPPAGE_PCT = 0 #0.0003     # 0.03% slippage applied to entry/exit (positive worsens outcome)

# Safety: smallest tradable unit (1 share)
MIN_SHARES = 1

# ======= Fee parameters (from the table you provided) =======
BROKERAGE_PCT = 0.0003       # 0.03% (decimal)
BROKERAGE_MIN_RUPEES = 20.0  # min fee cap per executed order (per side)
TRANSACTION_CHARGES_PCT = 0.0000297  # 0.00297% (NSE intraday)
SEBI_CHARGE_PER_CRORE = 10.0  # ₹10 per ₹1,00,00,000 -> decimal factor = 10 / 1e7 = 1e-6
SEBI_CHARGE_FACTOR = SEBI_CHARGE_PER_CRORE / 1e7  # equals 1e-6
GST_RATE = 0.18
STAMP_BUY_PCT = 0.00003      # 0.003% on buy side (or ₹300/crore)
STT_SELL_PCT = 0.00025       # 0.025% on sell side

# =========================
# Helpers
# =========================
def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

def to_ist_index(df):
    idx = pd.to_datetime(df.index)
    if idx.tz is None:
        idx = idx.tz_localize('UTC').tz_convert('Asia/Kolkata')
    else:
        idx = idx.tz_convert('Asia/Kolkata')
    df2 = df.copy()
    df2.index = idx
    return df2

def compute_session_vwap(df):
    df = df.copy()
    df['Date'] = df.index.date
    df['Time'] = df.index.time
    df['TP'] = (df['High'] + df['Low'] + df['Close']) / 3.0
    df['TPxV'] = df['TP'] * df['Volume']
    frames = []
    for d, grp in df.groupby('Date', sort=True):
        grp = grp.sort_index().copy()
        running_tpxv = 0.0
        running_vol = 0.0
        vwap_vals = []
        for tpxv, vol, t in zip(grp['TPxV'], grp['Volume'], grp['Time']):
            if t >= VWAP_SESSION_START:
                running_tpxv += tpxv
                running_vol += vol
                vwap_vals.append(running_tpxv / running_vol if running_vol != 0 else np.nan)
            else:
                vwap_vals.append(np.nan)
        grp['VWAP'] = vwap_vals
        frames.append(grp)
    out = pd.concat(frames).sort_index()
    out.drop(columns=['Date','Time','TPxV','TP'], inplace=True, errors='ignore')
    return out

def add_indicators(df):
    df = df.copy()
    df['EMA20'] = df['Close'].ewm(span=20, adjust=False).mean()
    ema_fast = df['Close'].ewm(span=48, adjust=False).mean()
    ema_slow = df['Close'].ewm(span=104, adjust=False).mean()
    df['MACD'] = ema_fast - ema_slow
    df['MACD_Signal'] = df['MACD'].ewm(span=36, adjust=False).mean()
    df['MACD_Hist'] = df['MACD'] - df['MACD_Signal']
    return df

def apply_slippage(price, side):
    if SLIPPAGE_PCT == 0:
        return price
    if side == 'buy':
        return price * (1.0 + SLIPPAGE_PCT)
    else:
        return price * (1.0 - SLIPPAGE_PCT)

def floor_shares(shares):
    s = math.floor(shares)
    return max(0, s)

import math

# =========================
# NEW: realistic intraday fee calculator
# =========================
def compute_intraday_fees(notional, side):
    """
    Compute intraday fees for a single executed side (buy or sell) following the table.

    Returns:
      total_fee (float), breakdown (dict)
    Breakdown includes:
      brokerage, transaction_charges, sebi, gst, stamp (buy only), stt (sell only)
    """
    # Brokerage: min(BROKERAGE_PCT * notional, BROKERAGE_MIN_RUPEES)
    raw_brokerage = BROKERAGE_PCT * notional
    brokerage = min(raw_brokerage, BROKERAGE_MIN_RUPEES)

    # Transaction charges (NSE)
    transaction_charges = TRANSACTION_CHARGES_PCT * notional

    # SEBI charges (₹10 per crore)
    sebi = SEBI_CHARGE_FACTOR * notional  # = 1e-6 * notional

    # GST: 18% on (brokerage + transaction_charges + sebi)
    gst_base = brokerage + transaction_charges + sebi
    gst = GST_RATE * gst_base

    # Stamp duty on buy side (0.003% or ₹300/crore)
    stamp = 0.0
    if side == 'buy':
        stamp = STAMP_BUY_PCT * notional

    # STT on sell side (0.025% on sell side)
    stt = 0.0
    if side == 'sell':
        stt = STT_SELL_PCT * notional

    total = brokerage + transaction_charges + sebi + gst + stamp + stt
    breakdown = {
        'brokerage': brokerage,
        'transaction_charges': transaction_charges,
        'sebi': sebi,
        'gst': gst,
        'stamp': stamp,
        'stt': stt,
        'total': total
    }
    return total, breakdown

def determine_exit_reason(entry_price, exit_price, tp_price, sl_price, vwap, ema):
    tol = 1e-6
    if tp_price is not None and abs(exit_price - tp_price) <= tol:
        return 'TP'
    if sl_price is not None and abs(exit_price - sl_price) <= tol:
        return 'SL'
    if (not np.isnan(vwap)) and abs(exit_price - vwap) <= 1e-6:
        return 'VWAP_BREACH'
    if (not np.isnan(ema)) and abs(exit_price - ema) <= 1e-6:
        return 'EMA_BREACH'
    return 'EXIT'

# =========================
# Core backtest (next-bar open + fees integrated)
# =========================
def backtest_ticker(ticker):
    print(f"\n--- Backtesting {ticker} (last ~60 days of {INTERVAL} bars) ---")
    raw = yf.download(ticker, period="60d", interval=INTERVAL, progress=False, threads=True, multi_level_index=False)
    if raw.empty:
        print(f"No data for {ticker}. Skipping.")
        return None, None, None

    df = to_ist_index(raw)
    df = compute_session_vwap(df)
    df = add_indicators(df)
    df = df.sort_index()

    buying_power = INITIAL_BALANCE * LEVERAGE
    cash = INITIAL_BALANCE
    position = None
    trades = []
    equity_curve = []
    scheduled_entry = None

    timestamps = list(df.index)
    for idx_pos, idx in enumerate(timestamps):
        row = df.loc[idx]
        bar_open = row['Open']; bar_high = row['High']; bar_low = row['Low']; bar_close = row['Close']
        bar_vwap = row.get('VWAP', np.nan); bar_ema = row.get('EMA20', np.nan)
        bar_time = idx.timetz().replace(tzinfo=None)

        # Execute scheduled entry at this bar's OPEN
        if scheduled_entry is not None and scheduled_entry['exec_index'] == idx and position is None:
            raw_entry_price = bar_open
            eff_entry_price = apply_slippage(raw_entry_price, 'buy')
            available_notional = buying_power * TRADE_SIZE_FRACTION
            raw_shares = available_notional / eff_entry_price
            shares = math.floor(raw_shares)
            if shares >= MIN_SHARES:
                notional = shares * eff_entry_price
                margin_required = notional / LEVERAGE
                borrowed = notional - margin_required
                entry_fee_total, entry_fee_break = compute_intraday_fees(notional, 'buy')
                # pay margin and entry fees (entry fees include stamp, brokerage, txn, sebi, gst)
                cash -= (margin_required + entry_fee_total)
                position = {
                    'entry_time': idx,
                    'entry_price_raw': raw_entry_price,
                    'entry_price_effective': eff_entry_price,
                    'shares': shares,
                    'notional': notional,
                    'margin_required': margin_required,
                    'borrowed': borrowed,
                    'entry_fees': entry_fee_break,
                    'entry_fees_total': entry_fee_total
                }
                trades.append({
                    'entry_time': position['entry_time'],
                    'exit_time': None,
                    'entry_price': position['entry_price_raw'],
                    'entry_price_effective': position['entry_price_effective'],
                    'exit_price': None,
                    'exit_price_effective': None,
                    'shares': position['shares'],
                    'notional': position['notional'],
                    'margin_required': position['margin_required'],
                    'borrowed': position['borrowed'],
                    'entry_fees_total': position['entry_fees_total'],
                    'entry_fees_breakdown': position['entry_fees'],
                    'exit_fees_total': None,
                    'exit_fees_breakdown': None,
                    'pnl_before_fees': None,
                    'pnl_after_fees': None,
                    'exit_reason': 'OPEN'
                })
            scheduled_entry = None

        # Intrabar exits
        if position is not None:
            tp_price = position['entry_price_effective'] + (PROFIT_TARGET_ABS / position['shares'])
            sl_price = position['entry_price_effective'] + (LOSS_LIMIT_ABS / position['shares'])
            triggered_prices = []

            if bar_high >= tp_price:
                triggered_prices.append(tp_price)
            if bar_low <= sl_price:
                triggered_prices.append(sl_price)
            if (not np.isnan(bar_vwap)) and bar_low <= bar_vwap:
                triggered_prices.append(bar_vwap)
            if (not np.isnan(bar_ema)) and bar_low <= bar_ema:
                triggered_prices.append(bar_ema)

            if triggered_prices:
                chosen_exit_price = max(triggered_prices)  # best for long
                eff_exit_price = apply_slippage(chosen_exit_price, 'sell')
                exit_notional = eff_exit_price * position['shares']
                exit_fee_total, exit_fee_break = compute_intraday_fees(exit_notional, 'sell')
                # when closing: proceeds = exit_notional
                # repay borrowed: cash had margin deducted at entry; now we return proceeds - borrowed minus exit fees
                cash += (exit_notional - position['borrowed'])
                cash -= exit_fee_total
                pnl_before = (chosen_exit_price - position['entry_price_effective']) * position['shares']
                pnl_after = pnl_before - position['entry_fees_total'] - exit_fee_total

                # fill last open trade
                for t in reversed(trades):
                    if t['exit_time'] is None and t['entry_time'] == position['entry_time']:
                        t['exit_time'] = idx
                        t['exit_price'] = chosen_exit_price
                        t['exit_price_effective'] = eff_exit_price
                        t['exit_fees_total'] = exit_fee_total
                        t['exit_fees_breakdown'] = exit_fee_break
                        t['pnl_before_fees'] = pnl_before
                        t['pnl_after_fees'] = pnl_after
                        t['exit_reason'] = determine_exit_reason(position['entry_price_effective'],
                                                               chosen_exit_price, tp_price, sl_price, bar_vwap, bar_ema)
                        break
                position = None

        # Forced EOD exit
        if position is not None and bar_time >= MARKET_CLOSE:
            chosen_exit_price = bar_close
            eff_exit_price = apply_slippage(chosen_exit_price, 'sell')
            exit_notional = eff_exit_price * position['shares']
            exit_fee_total, exit_fee_break = compute_intraday_fees(exit_notional, 'sell')
            cash += (exit_notional - position['borrowed'])
            cash -= exit_fee_total
            pnl_before = (chosen_exit_price - position['entry_price_effective']) * position['shares']
            pnl_after = pnl_before - position['entry_fees_total'] - exit_fee_total
            for t in reversed(trades):
                if t['exit_time'] is None and t['entry_time'] == position['entry_time']:
                    t['exit_time'] = idx
                    t['exit_price'] = chosen_exit_price
                    t['exit_price_effective'] = eff_exit_price
                    t['exit_fees_total'] = exit_fee_total
                    t['exit_fees_breakdown'] = exit_fee_break
                    t['pnl_before_fees'] = pnl_before
                    t['pnl_after_fees'] = pnl_after
                    t['exit_reason'] = 'MARKET_CLOSE'
                    break
            position = None

        # record equity
        if position is None:
            equity = cash
        else:
            market_value = position['shares'] * bar_close
            equity = cash + market_value - position['borrowed']
        equity_curve.append({'datetime': idx, 'equity': equity})

        # Evaluate entry signal on THIS bar and schedule for next bar open
        if (ENTRY_WINDOW_START <= bar_time <= ENTRY_WINDOW_END) and position is None:
            cond_vwap = (not np.isnan(bar_vwap)) and (bar_close > bar_vwap)
            cond_ema = (not np.isnan(row['EMA20'])) and (bar_close > row['EMA20'])
            cond_macd = (not np.isnan(row['MACD_Hist'])) and (row['MACD_Hist'] > 0)
            if cond_vwap and cond_ema and cond_macd:
                if idx_pos + 1 < len(timestamps):
                    scheduled_entry = {'exec_index': timestamps[idx_pos + 1]}

    # final safety close
    if position is not None:
        last_idx = df.index[-1]
        chosen_exit_price = df['Close'].iloc[-1]
        eff_exit_price = apply_slippage(chosen_exit_price, 'sell')
        exit_notional = eff_exit_price * position['shares']
        exit_fee_total, exit_fee_break = compute_intraday_fees(exit_notional, 'sell')
        cash += (exit_notional - position['borrowed'])
        cash -= exit_fee_total
        pnl_before = (chosen_exit_price - position['entry_price_effective']) * position['shares']
        pnl_after = pnl_before - position['entry_fees_total'] - exit_fee_total
        for t in reversed(trades):
            if t['exit_time'] is None and t['entry_time'] == position['entry_time']:
                t['exit_time'] = last_idx
                t['exit_price'] = chosen_exit_price
                t['exit_price_effective'] = eff_exit_price
                t['exit_fees_total'] = exit_fee_total
                t['exit_fees_breakdown'] = exit_fee_break
                t['pnl_before_fees'] = pnl_before
                t['pnl_after_fees'] = pnl_after
                t['exit_reason'] = 'FINAL_CLOSE'
                break
        position = None

    trades_df = pd.DataFrame(trades)
    equity_df = pd.DataFrame(equity_curve).set_index('datetime')

    # fill missing numeric cols
    if not trades_df.empty:
        trades_df.fillna(value={'pnl_before_fees': 0.0, 'pnl_after_fees': 0.0}, inplace=True)

    total_pnl = float(trades_df['pnl_after_fees'].sum()) if not trades_df.empty else 0.0
    wins = int((trades_df['pnl_after_fees'] > 0).sum()) if not trades_df.empty else 0
    losses = int((trades_df['pnl_after_fees'] <= 0).sum()) if not trades_df.empty else 0
    summary = {
        'ticker': ticker,
        'num_trades': len(trades_df),
        'total_pnl': total_pnl,
        'wins': wins,
        'losses': losses,
        'final_equity': float(equity_df['equity'].iloc[-1]) if not equity_df.empty else INITIAL_BALANCE
    }
    return trades_df, equity_df, summary

# =========================
# Runner
# =========================
if __name__ == "__main__":
    ensure_dir(SAVE_FOLDER)
    summaries = []
    for tk in TICKERS:
        trades_df, equity_df, summary = backtest_ticker(tk)
        if trades_df is None:
            continue
        safe = tk.replace('/', '_')
        trades_file = os.path.join(SAVE_FOLDER, f"trades_{safe}.csv")
        equity_file = os.path.join(SAVE_FOLDER, f"equity_{safe}.csv")
        trades_df.to_csv(trades_file, index=False)
        equity_df.to_csv(equity_file)
        print(f"Saved {trades_file} ({len(trades_df)} rows) and {equity_file} ({len(equity_df)} rows)")
        summaries.append(summary)

    if summaries:
        print("\n=== SUMMARY ===")
        print(pd.DataFrame(summaries).to_string(index=False))
    else:
        print("No results.")



--- Backtesting ICICIBANK.NS (last ~60 days of 15m bars) ---


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_10779/1844423098.py:184: FutureWarning: YF.download() has changed argument auto_adjust default to True
  raw = yf.download(ticker, period="60d", interval=INTERVAL, progress=False, threads=True, multi_level_index=False)


Saved ./backtest_results_with_fees/trades_ICICIBANK.NS.csv (205 rows) and ./backtest_results_with_fees/equity_ICICIBANK.NS.csv (1473 rows)

=== SUMMARY ===
      ticker  num_trades     total_pnl  wins  losses  final_equity
ICICIBANK.NS         205 -85113.840686    35     170 114886.159314


In [27]:
#!/usr/bin/env python3
"""
Intraday VWAP + EMA + MACD Backtest with Risk Controls (Pure Python Version)
---------------------------------------------------------------------------
BUY when:
    1. Close > Session VWAP
    2. Close > EMA(20)
    3. MACD Histogram (48,104,36) > 0
    4. (Optional) ADX > 20

EXIT when:
    1. Price < VWAP or EMA(20)
    2. Daily Profit >= ₹2000
    3. Daily Loss <= -₹500
    4. Stop after 2 consecutive losses per day
"""

import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta, time

# ==========================
# USER CONFIG
# ==========================
STOCKS = ["ICICIBANK.NS"]
CAPITAL = 100000        # Starting capital
MARGIN = 5              # Leverage 5x
PROFIT_TARGET = 2000    # Daily profit cap
LOSS_LIMIT = -500       # Daily loss limit
COMMISSION = 0.0005     # 0.05% per trade
SLIPPAGE = 0.0005       # 0.05% price impact
USE_ADX_FILTER = True   # Toggle ADX filter
USE_VOL_FILTER = False  # Toggle ATR-based volatility filter
ADX_THRESHOLD = 20
TIME_START = time(9, 45)
TIME_END = time(14, 0)
PERIOD = "15m"

# ==========================
# INDICATOR FUNCTIONS
# ==========================

def ema(series, period):
    return series.ewm(span=period, adjust=False).mean()

def vwap(high, low, close, volume, window=20):
    typical_price = (high + low + close) / 3
    return (typical_price * volume).rolling(window=window).sum() / volume.rolling(window=window).sum()

def macd_histogram(close, fast=48, slow=104, signal=36):
    ema_fast = ema(close, fast)
    ema_slow = ema(close, slow)
    macd_line = ema_fast - ema_slow
    signal_line = ema(macd_line, signal)
    return macd_line - signal_line

def adx(high, low, close, window=14):
    """Compute ADX (Average Directional Index) manually"""
    plus_dm = high.diff()
    minus_dm = low.diff().abs()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm < 0] = 0

    tr1 = high - low
    tr2 = (high - close.shift()).abs()
    tr3 = (low - close.shift()).abs()
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)

    atr = tr.rolling(window).mean()
    plus_di = 100 * (plus_dm.rolling(window).sum() / atr)
    minus_di = 100 * (minus_dm.rolling(window).sum() / atr)
    dx = (abs(plus_di - minus_di) / (plus_di + minus_di)) * 100
    adx = dx.rolling(window).mean()
    return adx

def atr(high, low, close, window=14):
    tr1 = high - low
    tr2 = (high - close.shift()).abs()
    tr3 = (low - close.shift()).abs()
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    return tr.rolling(window).mean()

# ==========================
# DATA FETCHING
# ==========================

def fetch_data(ticker):
    """Fetch recent 60 days of intraday data from Yahoo"""
    end = datetime.now()
    start = end - timedelta(days=59)
    df = yf.download(ticker, start=start, end=end, interval=PERIOD, progress=False, multi_level_index=False)
    if df.empty:
        print(f"[!] No data for {ticker}")
        return None

    # Handle timezone
    if df.index.tz is None:
        df = df.tz_localize("UTC").tz_convert("Asia/Kolkata")
    else:
        df = df.tz_convert("Asia/Kolkata")

    # Keep only Indian market hours
    df = df.between_time(TIME_START, TIME_END)
    return df

# ==========================
# INDICATOR CALCULATION
# ==========================

def compute_indicators(df):
    df["EMA20"] = ema(df["Close"], 20)
    df["VWAP"] = vwap(df["High"], df["Low"], df["Close"], df["Volume"], window=20)
    df["MACD_HIST"] = macd_histogram(df["Close"], fast=48, slow=104, signal=36)
    if USE_ADX_FILTER:
        df["ADX"] = adx(df["High"], df["Low"], df["Close"], window=14)
    if USE_VOL_FILTER:
        df["ATR"] = atr(df["High"], df["Low"], df["Close"], window=14)
    return df.dropna()

# ==========================
# BACKTEST ENGINE
# ==========================

def backtest(df, ticker):
    trades = []
    position = 0
    entry_price = 0
    pnl = 0
    total_pnl = 0
    loss_streak = 0
    last_date = None
    daily_pnl = 0

    for i in range(1, len(df)-1):
        row = df.iloc[i]
        next_row = df.iloc[i+1]
        date = row.name.date()

        # Reset daily vars
        if last_date != date:
            loss_streak = 0
            daily_pnl = 0
            last_date = date

        if loss_streak >= 2 or daily_pnl <= LOSS_LIMIT or daily_pnl >= PROFIT_TARGET:
            continue

        adx_ok = (not USE_ADX_FILTER) or (row.get("ADX", 25) > ADX_THRESHOLD)
        vol_ok = (not USE_VOL_FILTER) or (row.get("ATR", 1) > df["ATR"].median())

        # ENTRY CONDITION
        if (position == 0 and 
            row["Close"] > row["VWAP"] and 
            row["Close"] > row["EMA20"] and 
            row["MACD_HIST"] > 0 and 
            adx_ok and vol_ok):

            entry_price = next_row["Open"] * (1 + SLIPPAGE)
            size = int((CAPITAL * MARGIN) / entry_price)
            position = size
            entry_time = next_row.name
            trades.append({
                "Ticker": ticker,
                "EntryTime": entry_time,
                "EntryPrice": entry_price,
                "Size": size,
                "ExitTime": None,
                "ExitPrice": None,
                "PnL": None,
                "ExitReason": None
            })

        # EXIT CONDITION
        elif position > 0 and (row["Close"] < row["VWAP"] or row["Close"] < row["EMA20"]):
            exit_price = next_row["Open"] * (1 - SLIPPAGE)
            trade_pnl = (exit_price - entry_price) * position
            trade_pnl -= (entry_price + exit_price) * position * COMMISSION
            total_pnl += trade_pnl
            daily_pnl += trade_pnl
            trades[-1].update({
                "ExitTime": next_row.name,
                "ExitPrice": exit_price,
                "PnL": trade_pnl,
                "ExitReason": "Below VWAP/EMA"
            })
            if trade_pnl < 0:
                loss_streak += 1
            position = 0

    df_trades = pd.DataFrame(trades)
    return df_trades

# ==========================
# MAIN LOOP
# ==========================

all_trades = []

for symbol in STOCKS:
    print(f"Running backtest on {symbol} ...")
    data = fetch_data(symbol)
    if data is None or data.empty:
        continue
    data = compute_indicators(data)
    trades = backtest(data, symbol)
    all_trades.append(trades)

if all_trades:
    result = pd.concat(all_trades, ignore_index=True)
    result.to_csv("trades.csv", index=False)
    print("\n✅ Backtest completed. Trades saved to trades.csv")
    print(result.tail())
else:
    print("No trades executed.")

Running backtest on ICICIBANK.NS ...


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_10779/53281228.py:92: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start, end=end, interval=PERIOD, progress=False, multi_level_index=False)



✅ Backtest completed. Trades saved to trades.csv
          Ticker                 EntryTime   EntryPrice  Size  \
10  ICICIBANK.NS 2025-10-07 10:00:00+05:30  1378.388801   362   
11  ICICIBANK.NS 2025-10-09 10:45:00+05:30  1372.385801   364   
12  ICICIBANK.NS 2025-10-13 10:30:00+05:30  1382.490949   361   
13  ICICIBANK.NS 2025-10-14 10:00:00+05:30  1381.290276   361   
14  ICICIBANK.NS 2025-10-15 10:15:00+05:30  1390.494949   359   

                    ExitTime    ExitPrice           PnL      ExitReason  
10 2025-10-08 10:00:00+05:30  1376.811250  -1069.764725  Below VWAP/EMA  
11 2025-10-13 10:00:00+05:30  1377.510851   1365.037027  Below VWAP/EMA  
12 2025-10-13 10:45:00+05:30  1380.109649  -1358.298726  Below VWAP/EMA  
13 2025-10-14 11:15:00+05:30  1376.511351  -2222.974894  Below VWAP/EMA  
14 2025-10-17 14:00:00+05:30  1423.188074  11231.775970  Below VWAP/EMA  
